In [1]:
import numpy as np
import pandas as pd
import requests

In [ ]:
def get_data(base):
    
    url = f'{base}'
    result = requests.get(url)
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)
    
    # fix columns names
    records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()
    
    # Replace location.lat & location.lon
    column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='location' else a for a in column_names]
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    next_url = None
    
    # get next url
    if records_df.shape[0] != max_results: #added second cap to loop for truncation issue
        for l in links:
            if l['rel']=='next':
                next_url = l'
                
    return [records_df, next_url, column_names]

In [ ]:
#call the function 
df_new, url_next, column_names = get_data(BASE_URL,Res_Dataset)

# Create new EMPTy dataframe with columns
New_Res_Df = pd.DataFrame(columns=column_names)
New_Res_Df

# Create function to add API output to dataframe
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

# Get the rest of the Dataset
while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL,Res_Dataset, New_Res_Df.shape[0])
    New_Res_Df = add_to_dataframe(New_Res_Df, df_new)
    print(New_Res_Df.shape)
    if New_Res_Df.shape[0]>=9900: # Added for truncation issue
        break

In [5]:
r = requests.get(f'https://vicdata.vicroads.vic.gov.au/server/rest/services/SMOP/SMOP_Open_Data_040521/FeatureServer/1/query?outFields=*&where=1%3D1"ArithmeticError')

In [6]:
r

<Response [200]>

In [14]:
result = requests.get(f'https://vicdata.vicroads.vic.gov.au/server/rest/services/SMOP/SMOP_Open_Data_040521/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [17]:
result_geojson = requests.get(f'https://vicdata.vicroads.vic.gov.au/server/rest/services/SMOP/SMOP_Open_Data_040521/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson')

In [18]:
result_geojson

<Response [200]>